# Importando as Bibliotecas

In [31]:
import pandas as pd
import numpy as np
import string

from scipy import spatial

from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer

import time

# Lendo os Datasets

In [2]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_treino = []
lista_df_teste = []
for arquivo in arquivos:

    df_treino = pd.read_csv(f"Dados/Datasets/Treino/{arquivo}_treino.csv", dtype = {'ean_1': str, 'ean_2': str})
    df_teste = pd.read_csv(f"Dados/Datasets/Teste/{arquivo}_teste.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_treino.append(df_treino)
    lista_df_teste.append(df_teste)

# Aplicando o BoW

In [4]:
def remove_pontuacao(texto):
    
    texto_sp = texto.translate(str.maketrans('', '', string.punctuation))
    
    return texto_sp   


def formatar_entrada_bow(dados, mf = 1000):
    
    cv = CountVectorizer(
                         lowercase = True,
                         strip_accents = 'unicode',
                         max_features = mf
                        )

    cv.fit(dados)
    dados_transformados = cv.transform(dados).toarray()

    #X = matriz.fit_transform(dados).toarray()
    
    return cv, dados_transformados


def calcular_dis_cos(vetor_1, vetor_2):
                
    return (1 - spatial.distance.cosine(vetor_1, vetor_2))


def ordenar_resultado(res):

    indices = []
    valores = []

    for i in range(len(res)):
        
        ind = res[i].argsort()
        val = res[i][res[i].argsort()]

        indices.append( list(reversed(ind)) )
        valores.append( list(reversed(val)) )

    return indices, valores

In [29]:
def pipeline_cos(df, tam_max):
    
    # calculando o BoW do título
    cv_1, titulo_bow_1 = formatar_entrada_bow(df['titulo_pp_1'], mf = tam_max)
    cv_2, titulo_bow_2 = formatar_entrada_bow(df['titulo_pp_2'], mf = tam_max)

    distancia = []
    for i in range(len(titulo_bow_1)):
        distancia.append(calcular_dis_cos(titulo_bow_1[i], titulo_bow_2[i]))

    return distancia

In [33]:
flag_cos = True
if flag_cos == True:

    for nome, df_treino, df_teste in zip(arquivos, lista_df_treino, lista_df_teste):

        y_teste = df_teste["match"].to_list()

        # removendo a pontuação do título
        df_teste["titulo_pp_1"] = df_teste["titulo_1"].apply(lambda x: remove_pontuacao(x))
        df_teste["titulo_pp_2"] = df_teste["titulo_2"].apply(lambda x: remove_pontuacao(x))

        # calculando o tamanho máximo do título
        tam_max_1 = max(df_teste.apply(lambda row: len(row["titulo_pp_1"]), axis = 1))
        tam_max_2 = max(df_teste.apply(lambda row: len(row["titulo_pp_2"]), axis = 1))

        tam_max = tam_max_1 if tam_max_1 >= tam_max_2 else tam_max_2

        inicio_tempo = time.time()
        distancia = pipeline_cos(df_teste, tam_max)
        final_tempo = time.time()

        tempo = final_tempo - inicio_tempo

        y_pred = [1 if num >= 0.5 else 0 for num in distancia]

        report = classification_report(y_teste, y_pred, output_dict = True)
        df_resultado = pd.DataFrame(report).transpose()
        df_resultado['modelo'] = nome
        df_resultado['tempo'] = tempo

        df_resultado.to_csv(f'Dados/Resultados/Cos_05/{nome}_resultado.csv', index = True)

c:\Users\llvs2\anaconda3\lib\site-packages\scipy\spatial\distance.py:699: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
c:\Users\llvs2\anaconda3\lib\site-packages\scipy\spatial\distance.py:699: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
c:\Users\llvs2\anaconda3\lib\site-packages\scipy\spatial\distance.py:699: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
